<a id="item31"></a>


## Download and Clean Dataset


Let's start by importing the <em>pandas</em> and the Numpy libraries.


In [85]:
import pandas as pd
import numpy as np



<strong>The dataset is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used to make them. Ingredients include:</strong>

<strong>1. Cement</strong>

<strong>2. Blast Furnace Slag</strong>

<strong>3. Fly Ash</strong>

<strong>4. Water</strong>

<strong>5. Superplasticizer</strong>

<strong>6. Coarse Aggregate</strong>

<strong>7. Fine Aggregate</strong>


Let's download the data and read it into a dataframe.


In [86]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


#### Let's check how many data points we have.


In [87]:
concrete_data.shape

(1030, 9)

So, there are approximately 1000 samples to train our model on. Because of the few samples, we have to be careful not to overfit the training data.


Let's check the dataset for any missing values.


In [88]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [89]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data looks very clean and is ready to be used to build our model.


#### Split data into predictors and target


The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns.


In [90]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

<a id="item2"></a>


Let's do a quick sanity check of the predictors and the target dataframes.


In [91]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [92]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

Finally, the last step is to normalize the data by substracting the mean and dividing by the standard deviation.


Let's save the number of predictors to *n_cols* since we will need this number when building our network.


In [93]:
n_cols = predictors.shape[1] # number of predictors
n_cols

8

<a id="item1"></a>


<a id='item32'></a>


#### Let's go ahead and import the Keras library


In [94]:
import keras

Let's import the rest of the packages from the Keras library that we will need to build our regressoin model.


In [95]:
from keras.models import Sequential
from keras.layers import Dense

<a id='item33'></a>


## Build a Neural Network


Let's define a function that defines our regression model (1 hidden layer with 10 nodes and ReLu activation function) for us so that we can conveniently call it to create our model.


In [97]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

<a id="item4"></a>


<a id='item34'></a>


## Train and Test the Network


Let's call the function now to create our model.


In [98]:
# build the model
model = regression_model()

Next, we will train and test the model. We will leave out 30% of the data for testing (random split) and we will train the model for 50 epochs. We do this 50 times.


In [99]:
# normalize data

predictors_norm = (predictors - predictors.mean()) / predictors.std()
target= (target- target.mean()) / target.std()

In [100]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


In [101]:
MSElist = []
for k in range(50):
    X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size = 0.3, random_state = 20)
    model.fit(X_train, y_train, epochs=50, verbose=0)
    yhat = model.predict(X_test)
    mse = mean_squared_error(y_test, yhat)
    MSElist.append(mse)
print(MSElist)

[2.3219830844832585, 1.2254929441462197, 1.0017299628284335, 1.0358791911555763, 1.2836169701340225, 1.0918712755300322, 0.8434475157702883, 0.6369006453404588, 0.2662431381364449, 0.22294687281038955, 0.23338617227432573, 0.2420230151882581, 0.19085030356186175, 0.2082487960619789, 0.2668933137121324, 0.5326553513928849, 0.18396209094413696, 0.18460119919551857, 0.19394458059067637, 0.18430273568250818, 0.1844320304511239, 0.18452142420197937, 0.1814745331289607, 0.1849560109048159, 0.2126590378502774, 0.24401139067138353, 0.38225562260734125, 0.2689623963601276, 0.17684207123717932, 0.3123876689537038, 0.169296825538303, 0.24107750642028838, 0.16747341393413337, 0.17216686032203565, 0.1541382745034117, 0.1599314230945304, 0.15240766941826595, 0.1604851096382559, 0.15582720746311302, 0.16073534641051143, 0.16468414716093305, 0.15591476470798213, 0.15868131302311847, 0.2276610218917509, 0.17057775081248905, 0.14249971009017495, 0.14420798760299494, 0.17172313026417627, 0.15502654700767

In [102]:
from statistics import mean, stdev
print("Mean of the mean squared losses: ", mean(MSElist))
print("Standard deviation of the mean squared losses: ", stdev(MSElist))

Mean of the mean squared losses:  0.36036710242337583
Standard deviation of the mean squared losses:  0.41395238947585067


Note that the mean has decreased when deploying a network with 3 hidden layers (each consisting of 10 nodes + ReLu activation function) instead of deploying a network with 1 hidden layer (consisting of 10 nodes + ReLu activation function)